In [90]:
#import packages
import pandas as pd
import geopandas as gpd
import geojson
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, LogColorMapper, CategoricalColorMapper,
                          FixedTicker, BasicTickFormatter, LogTicker, FuncTickFormatter,
                          PrintfTickFormatter, BasicTicker, Legend, LegendItem)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

In [91]:
#import Affordable Housing file
housing = pd.read_csv("Housing.csv",sep=",")

housing

,Community Area Name,Community Area Number,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Portage Park,15,ARO,4812-15 W. Montrose Apts.,4812-15 W. Montrose Ave.,60641,630-694-6968,@properties,2,NaN,NaN,NaN,NaN,NaN
1,West Englewood,67,Multifamily,New West Englewood Homes,2109 W. 63rd St.,60636,773-434-4929,Interfaith Housing Corp.,12,NaN,NaN,NaN,NaN,NaN
2,Englewood,68,Multifamily,Antioch Homes II,301 W. Marquette Road,60621,773-994-4546,"Universal Management Service, Inc.",69,1.175445e+06,1.860492e+06,41.772564,-87.632419,"(41.7725637689, -87.6324193182)"
3,Washington Park,40,Senior HUD 202,St. Edmund's Corners,5556 S. Michigan Ave.,60637,773-667-7583,St. Edmund's Redevelopment Corp.,53,1.178070e+06,1.867952e+06,41.792975,-87.622569,"(41.7929745219, -87.6225685185)"
4,Humboldt Park,23,Multifamily,Nelson Mandela Apts.,526 N. Troy St.,60624,773-227-6332,Bickerdike Apts.,6,1.155238e+06,1.903559e+06,41.891173,-87.705338,"(41.8911727354, -87.7053383382)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,Near North Side,8,Multfamily,Parkside of Old Town IIB,1151 N. Cleveland Ave.,60622,312-944-8500,Holsten Real Estate Dev. Corp.,12,1.172709e+06,1.908000e+06,41.902990,-87.641043,"(41.902989721, -87.6410428925)"
424,Near South Side,33,Multifamily,Hilliard Homes Phase I,2030 S. State St.,60616,312-225-7610,Holsten Real Estate Dev. Corp.,170,1.176605e+06,1.890374e+06,41.854535,-87.627268,"(41.854535033, -87.6272677388)"
425,Washington Park,40,Multifamily,St. Edmund's Commons,110 E. 60th St.,60637,773-420-3328,St. Edmund's Redevelopment Corp.,54,1.178316e+06,1.865303e+06,41.785701,-87.621749,"(41.7857009548, -87.6217485015)"
426,Near West Side,28,Senior,Loomis Court,1314 W. 15th St.,60608,312-850-4128,CHA,68,1.167767e+06,1.892903e+06,41.861670,-87.659631,"(41.8616698673, -87.659631083)"


In [92]:
#Group housing data by neighborhood and count number of addresses 

housing_grouped = housing.groupby(["Community Area Name"]).count()[["Address"]].reset_index()

housing_grouped

,Community Area Name,Address
0,Albany Park,3
1,Ashburn,1
2,Auburn Gresham,6
3,Austin,11
4,Avalon Park,1
...,...,...
58,West Lawn,1
59,West Pullman,3
60,West Ridge,5
61,West Town,23


In [93]:
#import shapefile with neighborhood boundaries
map_nbhoods = gpd.read_file('Neighborhoods.shp')

map_nbhoods

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
1,Printers Row,PRINTERS ROW,2.162138e+06,6864.247156,"POLYGON ((-87.62761 41.87437, -87.62760 41.873..."
2,United Center,UNITED CENTER,3.252051e+07,23101.363745,"POLYGON ((-87.66707 41.88885, -87.66707 41.888..."
3,Sheffield & DePaul,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745,"POLYGON ((-87.65833 41.92166, -87.65835 41.922..."
4,Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351,"POLYGON ((-87.74060 41.88782, -87.74060 41.887..."
...,...,...,...,...,...
93,Belmont Cragin,"BELMONT CRAGIN,HERMOSA",1.090994e+08,43311.706886,"POLYGON ((-87.74143 41.91698, -87.74141 41.916..."
94,Austin,AUSTIN,1.700378e+08,55473.345911,"POLYGON ((-87.75620 41.91547, -87.75588 41.915..."
95,Gold Coast,GOLD COAST,7.165706e+06,13685.479377,"POLYGON ((-87.62646 41.91147, -87.62640 41.911..."
96,Boystown,BOYSTOWN,3.365779e+06,9780.268985,"POLYGON ((-87.64878 41.93999, -87.64927 41.939..."


In [94]:
map_nbhoods.dtypes #check to be sure it's a geodataframe

pri_neigh       object
sec_neigh       object
shape_area     float64
shape_len      float64
geometry      geometry
dtype: object

In [95]:
'''Find Mismatched Neighorhoods'''

#merge neighborhood data with housing data based on neighborhood name
housing_merged = pd.merge(map_nbhoods,housing_grouped,
                         left_on='pri_neigh',right_on='Community Area Name',
                         how='outer',indicator=True)



housing_merged

,pri_neigh,sec_neigh,shape_area,shape_len,geometry,Community Area Name,Address,_merge
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816...",Grand Boulevard,27.0,both
1,Printers Row,PRINTERS ROW,2.162138e+06,6864.247156,"POLYGON ((-87.62761 41.87437, -87.62760 41.873...",NaN,NaN,left_only
2,United Center,UNITED CENTER,3.252051e+07,23101.363745,"POLYGON ((-87.66707 41.88885, -87.66707 41.888...",NaN,NaN,left_only
3,Sheffield & DePaul,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745,"POLYGON ((-87.65833 41.92166, -87.65835 41.922...",NaN,NaN,left_only
4,Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351,"POLYGON ((-87.74060 41.88782, -87.74060 41.887...",Humboldt Park,36.0,both
...,...,...,...,...,...,...,...,...
102,NaN,NaN,NaN,NaN,None,Near North Side,11.0,right_only
103,NaN,NaN,NaN,NaN,None,Near West Side,17.0,right_only
104,NaN,NaN,NaN,NaN,None,South Lawndale,3.0,right_only
105,NaN,NaN,NaN,NaN,None,West Englewood,3.0,right_only


In [96]:
#find housing units with mismatched neighborhoods
right_only = housing_merged.loc[housing_merged['_merge'] == 'right_only']

right_only

,pri_neigh,sec_neigh,shape_area,shape_len,geometry,Community Area Name,Address,_merge
98,NaN,NaN,NaN,NaN,None,East Garfield Park,10.0,right_only
99,NaN,NaN,NaN,NaN,None,East Garfiled Park,1.0,right_only
100,NaN,NaN,NaN,NaN,None,Greater Grand Crossing,2.0,right_only
101,NaN,NaN,NaN,NaN,None,Lakeview,9.0,right_only
102,NaN,NaN,NaN,NaN,None,Near North Side,11.0,right_only
103,NaN,NaN,NaN,NaN,None,Near West Side,17.0,right_only
104,NaN,NaN,NaN,NaN,None,South Lawndale,3.0,right_only
105,NaN,NaN,NaN,NaN,None,West Englewood,3.0,right_only
106,NaN,NaN,NaN,NaN,None,West Garfield Park,3.0,right_only


In [97]:
'''Clean Housing Data'''

#fix typos
housing.replace({'East Garfiled Park':'East Garfield Park'},inplace=True)
housing.replace({'Lakeview':'Lake View'},inplace=True)

#match up mismatched neighborhoods
housing.replace({'West Englewood':'Englewood',
                    'Near West Side':'Little Italy, UIC',
                    'Near North Side':'River North',
                    'East Garfield Park':'Garfield Park',
                    'West Garfield Park':'Garfield Park',
                    'Greater Grand Crossing':'Grand Crossing',
                    'South Lawndale':'Little Village'},inplace=True)

#drop rows with non-affordable housing
luxury = housing[(housing["Property Type"] == 'ARO')].index
housing.drop(luxury, inplace=True)

In [111]:
#Group cleaned housing data by neighborhood and count number of addresses 

housing_cleaned = housing.groupby(["Community Area Name"]).count()[["Address"]].reset_index()

In [99]:
#merge cleaned neighborhood data with housing data based on neighborhood name
housing_nbhoods = pd.merge(map_nbhoods,housing_cleaned,
                         left_on='pri_neigh',right_on='Community Area Name',
                         how='outer',indicator=True)

In [100]:
#replace NaN (neighborhoods with no matching housing units) with 0
housing_nbhoods['Address'] = housing_nbhoods['Address'].fillna(0)

housing_nbhoods

,pri_neigh,sec_neigh,shape_area,shape_len,geometry,Community Area Name,Address,_merge
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816...",Grand Boulevard,27.0,both
1,Printers Row,PRINTERS ROW,2.162138e+06,6864.247156,"POLYGON ((-87.62761 41.87437, -87.62760 41.873...",NaN,0.0,left_only
2,United Center,UNITED CENTER,3.252051e+07,23101.363745,"POLYGON ((-87.66707 41.88885, -87.66707 41.888...",NaN,0.0,left_only
3,Sheffield & DePaul,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745,"POLYGON ((-87.65833 41.92166, -87.65835 41.922...",NaN,0.0,left_only
4,Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351,"POLYGON ((-87.74060 41.88782, -87.74060 41.887...",Humboldt Park,36.0,both
...,...,...,...,...,...,...,...,...
93,Belmont Cragin,"BELMONT CRAGIN,HERMOSA",1.090994e+08,43311.706886,"POLYGON ((-87.74143 41.91698, -87.74141 41.916...",Belmont Cragin,4.0,both
94,Austin,AUSTIN,1.700378e+08,55473.345911,"POLYGON ((-87.75620 41.91547, -87.75588 41.915...",Austin,11.0,both
95,Gold Coast,GOLD COAST,7.165706e+06,13685.479377,"POLYGON ((-87.62646 41.91147, -87.62640 41.911...",NaN,0.0,left_only
96,Boystown,BOYSTOWN,3.365779e+06,9780.268985,"POLYGON ((-87.64878 41.93999, -87.64927 41.939...",NaN,0.0,left_only


In [101]:
#read dataframe as geodataframe

gdf = gpd.GeoDataFrame(housing_nbhoods, geometry='geometry')

#convert geodataframe to geojson
geosource = GeoJSONDataSource(geojson=gdf.to_json())

In [102]:
'''Create Affordable Housing by Neighborhood Map'''

# Define color palettes
palette = brewer['BuGn'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LogColorMapper that exponentially maps numbers in a range, into a sequence of colors.
color_mapper = LogColorMapper(palette = palette, low = 0, high = 40)

# Define custom tick labels for color bar.
tick_labels = {1.35:'0',2.5:'1-2',4.6: '3-5',8.5: '6-10', 16:'11-19',
 30:'20+'}

# Create color bar
color_bar = ColorBar(title = 'Number of Housing Units',
                     color_mapper = color_mapper, 
                     label_standoff = 6,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0),
                     orientation = 'horizontal',
                     ticker=FixedTicker(num_minor_ticks=0,
                                        ticks=[1.35,2.5,4.6,8.5,16,30]),
                     major_label_overrides = tick_labels,
                     major_tick_line_color = None,
                     major_label_text_align = 'center')

# Create figure object
p = figure(title = 'Affordable Housing per Neighborhood in Chicago')


# Add patch renderer to figure
neighborhoods = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Address',
                                 'transform' : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [neighborhoods],
                      tooltips = [('Neighborhood','@pri_neigh'),
                                  ('No. of Housing Units','@Address')]))

#remove axes, axis labels, and grid lines
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '0pt'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Specify layout
p.add_layout(color_bar, 'below')

show(p)

In [103]:
#read L_stops data
L_Stops = pd.read_csv('L_Stops.csv',sep=",")

L_Stops

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,30106,S,Wilson (95th-bound),Wilson,Wilson (Red & Purple Lines),40540,True,True,False,False,False,False,False,False,False,False,"(41.964273, -87.657588)"
296,30383,N,Washington/Wabash (Outer Loop),Washington/Wabash,"Washington/Wabash (Brown, Green, Orange, Purpl...",41700,True,False,False,True,True,False,False,False,False,False,"(41.88322, -87.626189)"
297,30385,S,Wilson (Loop-bound),Wilson,Wilson (Red & Purple Lines),40540,True,False,False,False,False,False,True,False,False,False,"(41.964273, -87.657588)"
298,30033,W,Ashland (Harlem-54th/Cermak-bound),Ashland,Ashland (Green & Pink Lines),40170,True,False,False,True,False,False,False,False,True,False,"(41.885269, -87.666969)"


In [104]:
#Need to convert Location to Latitude and Longitude columns
new = L_Stops["Location"].str.split(",", n = 1, expand = True) 

#remove parentheses
new[0] = new[0].str.replace("(","") 
new[1] = new[1].str.replace(")","")

#convert type from string to float
new[0]= new[0].astype(float) 
new[1]= new[1].astype(float)

#split into 2 columns
L_Stops["Latitude"] = new[0]
L_Stops["Longitude"] = new[1]

In [105]:
#convert Latitude and Longitude to geometry datatype

GeoStops = gpd.GeoDataFrame(
    L_Stops, geometry=gpd.points_from_xy(L_Stops.Longitude, L_Stops.Latitude))

GeoStops

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Latitude,Longitude,geometry
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)",41.857908,-87.669147,POINT (-87.66915 41.85791)
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)",41.857908,-87.669147,POINT (-87.66915 41.85791)
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)",41.829353,-87.680622,POINT (-87.68062 41.82935)
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)",41.829353,-87.680622,POINT (-87.68062 41.82935)
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)",41.831677,-87.625826,POINT (-87.62583 41.83168)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,30106,S,Wilson (95th-bound),Wilson,Wilson (Red & Purple Lines),40540,True,True,False,False,False,False,False,False,False,False,"(41.964273, -87.657588)",41.964273,-87.657588,POINT (-87.65759 41.96427)
296,30383,N,Washington/Wabash (Outer Loop),Washington/Wabash,"Washington/Wabash (Brown, Green, Orange, Purpl...",41700,True,False,False,True,True,False,False,False,False,False,"(41.88322, -87.626189)",41.883220,-87.626189,POINT (-87.62619 41.88322)
297,30385,S,Wilson (Loop-bound),Wilson,Wilson (Red & Purple Lines),40540,True,False,False,False,False,False,True,False,False,False,"(41.964273, -87.657588)",41.964273,-87.657588,POINT (-87.65759 41.96427)
298,30033,W,Ashland (Harlem-54th/Cermak-bound),Ashland,Ashland (Green & Pink Lines),40170,True,False,False,True,False,False,False,False,True,False,"(41.885269, -87.666969)",41.885269,-87.666969,POINT (-87.66697 41.88527)


In [106]:
#drop Purple Line column
GeoStops.drop(columns="P", inplace=True)

In [107]:
'''Add counter for number of connecting lines per station'''

#Convert boolean to int
GeoStops["RED"] = GeoStops["RED"].astype(int)
GeoStops["BLUE"] = GeoStops["BLUE"].astype(int)
GeoStops["G"] = GeoStops["G"].astype(int)
GeoStops["Y"] = GeoStops["Y"].astype(int)
GeoStops["Pexp"] = GeoStops["Pexp"].astype(int)
GeoStops["Pnk"] = GeoStops["Pnk"].astype(int)
GeoStops["O"] = GeoStops["O"].astype(int)
GeoStops["BRN"] = GeoStops["BRN"].astype(int)

#add a column summing the number of lines that connect at each stop
GeoStops['Num_Lines'] = GeoStops[{"RED","BLUE","G","BRN","Pexp","Y","Pnk","O"}].sum(axis=1)
GeoStops_Lines = GeoStops.copy()

GeoStops_Lines

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,Pexp,Y,Pnk,O,Location,Latitude,Longitude,geometry,Num_Lines
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,0,0,0,0,0,0,1,0,"(41.857908, -87.669147)",41.857908,-87.669147,POINT (-87.66915 41.85791),1
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,0,0,0,0,0,0,1,0,"(41.857908, -87.669147)",41.857908,-87.669147,POINT (-87.66915 41.85791),1
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,0,0,0,0,0,0,0,1,"(41.829353, -87.680622)",41.829353,-87.680622,POINT (-87.68062 41.82935),1
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,0,0,0,0,0,0,0,1,"(41.829353, -87.680622)",41.829353,-87.680622,POINT (-87.68062 41.82935),1
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,0,0,1,0,0,0,0,0,"(41.831677, -87.625826)",41.831677,-87.625826,POINT (-87.62583 41.83168),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,30106,S,Wilson (95th-bound),Wilson,Wilson (Red & Purple Lines),40540,True,1,0,0,0,0,0,0,0,"(41.964273, -87.657588)",41.964273,-87.657588,POINT (-87.65759 41.96427),1
296,30383,N,Washington/Wabash (Outer Loop),Washington/Wabash,"Washington/Wabash (Brown, Green, Orange, Purpl...",41700,True,0,0,1,1,0,0,0,0,"(41.88322, -87.626189)",41.883220,-87.626189,POINT (-87.62619 41.88322),2
297,30385,S,Wilson (Loop-bound),Wilson,Wilson (Red & Purple Lines),40540,True,0,0,0,0,1,0,0,0,"(41.964273, -87.657588)",41.964273,-87.657588,POINT (-87.65759 41.96427),1
298,30033,W,Ashland (Harlem-54th/Cermak-bound),Ashland,Ashland (Green & Pink Lines),40170,True,0,0,1,0,0,0,1,0,"(41.885269, -87.666969)",41.885269,-87.666969,POINT (-87.66697 41.88527),2


In [108]:
#drop rows with an extra direction at the ends of lines

end_lines = GeoStops_Lines[(GeoStops_Lines["STOP_ID"] == 30077) | #Forest Park end of Blue Line
                           (GeoStops_Lines["STOP_ID"] == 30171) | #O'Hare end of Blue Line
                           (GeoStops_Lines["STOP_ID"] == 30249) | #End of Brown Line
                           (GeoStops_Lines["STOP_ID"] == 30182) | #End of Orange Line
                           (GeoStops_Lines["STOP_ID"] == 30203) | #End of Purple Line
                           (GeoStops_Lines["STOP_ID"] == 30089) | #95th end of Red Line
                           (GeoStops_Lines["STOP_ID"] == 30173) | #Howard end of Red Line
                           (GeoStops_Lines["STOP_ID"] == 30026) | #end of Yellow Line
                           (GeoStops_Lines["STOP_ID"] == 30139) | #Cottage Grove end of Green Line
                           (GeoStops_Lines["STOP_ID"] == 30057) | #Ashland end of Green Line
                           (GeoStops_Lines["STOP_ID"] == 30114) | #end of Pink Line
                           (GeoStops_Lines["STOP_ID"] == 30004)].index #Harlem end of Green Line

GeoStops_Lines.drop(end_lines, inplace=True)

In [109]:
#Group the number of lines by directions per station
Grouped_Stops = GeoStops_Lines.groupby(["STATION_DESCRIPTIVE_NAME","MAP_ID","Latitude","Longitude"]).sum()[["Num_Lines"]].reset_index()

Grouped_Stops

,STATION_DESCRIPTIVE_NAME,MAP_ID,Latitude,Longitude,Num_Lines
0,18th (Pink Line),40830,41.857908,-87.669147,2
1,35th-Bronzeville-IIT (Green Line),41120,41.831677,-87.625826,2
2,35th/Archer (Orange Line),40120,41.829353,-87.680622,2
3,43rd (Green Line),41270,41.816462,-87.619021,2
4,47th (Green Line),41080,41.809209,-87.618826,2
...,...,...,...,...,...
140,Western (Blue Line - O'Hare Branch),40670,41.916157,-87.687364,2
141,Western (Brown Line),41480,41.966163,-87.688502,2
142,Western (Orange Line),40310,41.804546,-87.684019,2
143,Western (Pink Line),40740,41.854225,-87.685129,2


In [115]:
'''Create Affordable Housing by Neighborhood with L Stops Map'''

#boundaries = gpd.read_file('Boundaries.geojson')
geosource = GeoJSONDataSource(geojson=gdf.to_json())

# Define color palettes
palette = brewer['BuGn'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LogColorMapper that exponentially maps numbers in a range, into a sequence of colors.
color_mapper = LogColorMapper(palette = palette, low = 0, high = 40)

# Define custom tick labels for color bar.
tick_labels = {1.35:'0',2.5:'1-2',4.6: '3-5',8.5: '6-10', 16:'11-19',
 30:'20+'}


# Create color bar.
color_bar = ColorBar(title = 'Number of Affordable Housing Units',
                     color_mapper = color_mapper, 
                     label_standoff = 6,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0),
                     orientation = 'horizontal',
                     ticker=FixedTicker(num_minor_ticks=0,
                                        ticks=[1.35,2.5,4.6,8.5,16,30]),
                     major_label_overrides = tick_labels,
                     major_tick_line_color = None,
                     major_label_text_align = 'center')

# Create figure object.
p = figure(title = 'Affordable Housing Rapid Transit Access in Chicago')


# Add patch renderer for neighborhood boundaries
neighborhoods = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Address',
                                 'transform' : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Add scatter renderer for stops
stops = p.scatter(Grouped_Stops["Longitude"],Grouped_Stops["Latitude"],
                  size=Grouped_Stops['Num_Lines']*2.5, #scaled so differences b/w number of lines/station is visible
                  line_color="#FF0000", 
                  fill_color="#FF0000",
                  fill_alpha=0.05)

#Add Legend that varies by size
legend1 = Legend(items=[
    LegendItem(label='1 connection', renderers=[stops])
    ],glyph_height=19, glyph_width=6, location=(22,85), border_line_color = None, label_standoff=16,
                title='L Stations',title_text_align="left", title_standoff=15)

legend2 = Legend(items=[
    LegendItem(label='4 connections', renderers=[stops])],glyph_height=25, glyph_width=25,
                      location=(13,50), border_line_color = None, label_standoff=7)

legend3 = Legend(items=[
    LegendItem(label='8 connections', renderers=[stops])],glyph_height=50, glyph_width=50,
                      location=(1,1), border_line_color = None, label_standoff=-5)

p.add_layout(legend1)
p.add_layout(legend2)
p.add_layout(legend3)

# Create hover tool
p.add_tools(HoverTool(renderers = [neighborhoods],
                      tooltips = [('Neighborhood','@pri_neigh'),
                                  ('No. of Housing Units','@Address')]))


#remove axes, axis labels, and grid lines
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '0pt'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


# Specify layout
p.add_layout(color_bar, 'below')

show(p)